In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual


import numpy as np
import matplotlib.pyplot as plt
import imageio

from imutils.video import VideoStream
from imutils.video import FPS
import argparse
import imutils
import time
import cv2

import matplotlib

### Optionally enable interactive tools (needed for clicking enabled coordinate getting tools)

In [ ]:
%matplotlib tk


# Helper functions and classes

In [ ]:


box_size = np.array([
    2.52, # Mokille pain
    1.905, # Rantaan pain
    1.115, # Korkeus
])

box_size = np.array([
    1.905, # Rantaan pain
    2.52, # Mokille pain
    1.115, # Korkeus
])

In [ ]:
def get_line(coordinate_1, coordinate_2):
    x_diff, y_diff = coordinate_1 - coordinate_2
    angle = y_diff / x_diff
    c = coordinate_1[1] - coordinate_1[0] * angle

    return angle, c

# y = x * angle + c

In [ ]:
def get_intersection(c1, c2, c3, c4):
    angle1, c1 = get_line(c1, c2)
    angle2, c2 = get_line(c3, c4)

    x = (c1 - c2) / (angle2 - angle1)
    y = x * angle1 + c1
    
    return np.array([x, y])

In [ ]:
def get_axis_distance_multiplier(horizon_point, box_point_1, box_point_2, intersection_point):
    #print(
    #    np.linalg.norm(horizon_point - box_point_1),
    #    np.linalg.norm(horizon_point - box_point_2),
    #    np.linalg.norm(horizon_point - intersection_point),
    #)
    #print(
    #    1 / np.power(np.linalg.norm(horizon_point - box_point_1), 2),
    #    1 / np.power(np.linalg.norm(horizon_point - box_point_2), 2),
    #    1 / np.power(np.linalg.norm(horizon_point - intersection_point), 2),
    #)
    
    y_distance_point_1 = 1 / np.power(np.linalg.norm(horizon_point - box_point_1), 2)
    y_distance_point_2 = 1 / np.power(np.linalg.norm(horizon_point - box_point_2), 2)
    y_distance_intersection_point = 1 / np.power(np.linalg.norm(horizon_point - intersection_point), 2)
    
    multiplier = (y_distance_point_1 - y_distance_intersection_point) / (y_distance_point_1 - y_distance_point_2)
    
    return multiplier


In [ ]:


class LocationGetter:
    def __init__(self, coordinates):
        c = np.array(coordinates)
        self.coordinates = c
        self.c = c

        # Intersections of horizon lines
        p1 = get_intersection(c[0], c[1], c[3], c[4])
        p2 = get_intersection(c[1], c[2], c[4], c[5])
        p3 = get_intersection(c[0], c[3], c[2], c[5])
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

        # Intersections of lines between horizon lines and horizon lines
        self.top = get_intersection(p1, p2, c[1], c[4])
        self.left = get_intersection(p1, p3, c[4], c[5])
        self.right = get_intersection(p3, p2, c[3], c[4])

        # Distance multipliers
        multiplier_1 = get_axis_distance_multiplier(p2, c[4], c[5], self.left)
        multiplier_2 = get_axis_distance_multiplier(p1, c[4], c[3], self.right)
        multiplier_3 = get_axis_distance_multiplier(p3, c[4], c[1], self.top)
        multipliers = np.array([multiplier_1, multiplier_2, multiplier_3])

        # Coordinates
        self.real_life_coordinates = multipliers * box_size

    def get(self):
        return self.real_life_coordinates
    
    def plot(self, image):
        height, width, _ = image.shape
        
        #plt.figure(figsize=(20, 20))

        plt.imshow(image)

        left = -99999
        right = width + 99999

        coordinates = self.coordinates
        
        linewidth = 2
        
        angle, c = get_line(coordinates[0], coordinates[1])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)
        angle, c = get_line(coordinates[1], coordinates[2])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)
        angle, c = get_line(coordinates[3], coordinates[4])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)
        angle, c = get_line(coordinates[4], coordinates[5])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)

        angle, c = get_line(coordinates[0], coordinates[3])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)
        angle, c = get_line(coordinates[1], coordinates[4])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)
        angle, c = get_line(coordinates[2], coordinates[5])
        plt.plot([left, right], [left * angle + c, right * angle + c], c='blue', linewidth=linewidth)

        

        plt.plot([self.p1[0], self.p2[0]], [self.p1[1], self.p2[1]], c='white', linewidth=linewidth)
        plt.plot([self.p2[0], self.p3[0]], [self.p2[1], self.p3[1]], c='white', linewidth=linewidth)
        plt.plot([self.p1[0], self.p3[0]], [self.p1[1], self.p3[1]], c='white', linewidth=linewidth)
        
        
        plt.scatter(*self.p1, s=100, c='white', marker='o', edgecolors='b')
        plt.scatter(*self.p2, s=100, c='white', marker='o', edgecolors='b')
        plt.scatter(*self.p3, s=100, c='white', marker='o', edgecolors='b')
        for index, coordinate in enumerate(self.coordinates):
            x, y = coordinate
            letter = 'ABCDEFG'[index]
            title = '{} / {}\n({}, {})'.format(letter, index, x, y)
            plt.scatter(*coordinate, s=100, c='white', marker='o', edgecolors='b')
            
            #plt.annotate(
            #    title,
            #    xy=(x - 120, y - 70),
            #    #xycoords='data',
            #    #xytext=(x / width, y / height),textcoords='figure fraction',
            #    #arrowprops=dict(arrowstyle="->", color='white'),
            #    color='black',
            #    bbox=dict(
            #        #boxstyle="rarrow,pad=0.3",
            #        #boxstyle='pad=1',
            #        fc="white", 
            #        ec="black",
            #        lw=2
            #    )
            #)
        
        def plot_axis(p1, p2, distance):
            plt.plot([p1[0], p2[0]], [p1[1], p2[1]], c='magenta', linewidth=4)
            plt.scatter(*p2, s=200, c='white', marker='o', edgecolors='b')
            x, y = p2
            title = '{:.2f}m'.format(distance)
            if x > 0 and x < width and y > 0 and y < height:
                plt.annotate(
                    title,
                    xy=(x - 120/2, y - 70/2),
                    color='black',
                    bbox=dict(
                        #boxstyle="rarrow,pad=0.3",
                        #boxstyle='pad=1',
                        fc="white", 
                        ec="black",
                        lw=2
                    )
                )
            #plt.ylim(height, 0)
            #plt.xlim(0, width)

        
        plot_axis(self.c[4], self.top, self.real_life_coordinates[2])
        plot_axis(self.c[4], self.left, self.real_life_coordinates[0])
        plot_axis(self.c[4], self.right, self.real_life_coordinates[1])
        
        #plt.plot([self.c[4][0], self.top[0]], [self.c[4][1], self.top[1]], c='magenta', linewidth=4)
        #plt.scatter(*self.top, s=200, c='magenta', marker='o', edgecolors='b')
        #plt.plot([self.c[4][0], self.left[0]], [self.c[4][1], self.left[1]], c='magenta', linewidth=4)
        #plt.scatter(*self.left, s=200, c='magenta', marker='o', edgecolors='b')
        #plt.plot([self.c[4][0], self.right[0]], [self.c[4][1], self.right[1]], c='magenta', linewidth=4)
        #plt.scatter(*self.right, s=200, c='magenta', marker='o', edgecolors='b')
        
        #plt.ylim(max(self.p1[1], self.p2[1], self.p3[1], height), min(self.p1[1], self.p2[1], self.p3[1], 0))
        #plt.xlim(min(self.p1[0], self.p2[0], self.p3[0]), max(self.p1[0], self.p2[0], self.p3[0]))
        plt.ylim(height, 0)
        plt.xlim(0, width)


# Testing all steps with images

In [ ]:
image = imageio.imread('img1.jpg')
height, width, _ = image.shape

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(image)

In [ ]:
#all_coordinates = []

all_coordinates = [
    [
        (1570, 1561),
        (2509, 1676),
        (3242, 1544),
        (1585, 2213),
        (2485, 2475),
        (3199, 2122)
    ],
    [
        (1879, 1320),
        (2496, 1342),
        (2701, 1314),
        (1873, 1709),
        (2477, 1753),
        (2686, 1643)
    ],
    [
        (1970, 1110),
        (2614, 1123),
        (2813, 1185),
        (1965, 1506),
        (2581, 1546),
        (2784, 1525)
    ]
]


In [ ]:
'''
image = imageio.imread('img3.jpg')

plt.plot(figsize=(10, 10))
ax = plt.gca()
fig = plt.gcf()
implot = ax.imshow(image)

coordinates = []

def onclick(event):
    #print('onclick')
    if event.xdata != None and event.ydata != None:
        coordinates.append((event.xdata, event.ydata))
        print('added')
        #print(event.xdata, event.ydata)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()
'''
pass

In [ ]:
#all_coordinates += coordinates

In [ ]:
#coordinates = np.array([
#    (1574, 1560),
#    (2510, 1678),
#    (3237, 1543),
#    (1591, 2232),
#    (2482, 2474),
#    (3191, 2123),
#])

coordinates = np.array(all_coordinates[0])

In [ ]:
plt.figure(figsize=(20, 20))

plt.imshow(image)


for index, coordinate in enumerate(coordinates):
    x, y = coordinate
    letter = 'ABCDEFG'[index]
    title = '{} / {}\n({}, {})'.format(letter, index, x, y)
    plt.annotate(
        title,
        xy=(x - 120, y - 70),
        #xycoords='data',
        #xytext=(x / width, y / height),textcoords='figure fraction',
        #arrowprops=dict(arrowstyle="->", color='white'),
        color='black',
        bbox=dict(
            #boxstyle="rarrow,pad=0.3",
            #boxstyle='pad=1',
            fc="white", 
            ec="black",
            lw=2
        )
    )
    plt.scatter(*coordinate, s=100, c='white', marker='o', edgecolors='b')

# plt.show()


In [ ]:
plt.figure(figsize=(20, 20))

plt.imshow(image)

for index, coordinate in enumerate(coordinates):
    x, y = coordinate
    letter = 'ABCDEFG'[index]
    title = '{} / {}\n({}, {})'.format(letter, index, x, y)
    plt.scatter(*coordinate, s=100, c='white', marker='o', edgecolors='b')

left = -99999
right = width + 99999
    
angle, c = get_line(coordinates[0], coordinates[1])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[2])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[3], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[4], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

angle, c = get_line(coordinates[0], coordinates[3])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[2], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

plt.ylim(height, 0)
plt.xlim(0, width)

In [ ]:
p1 = get_intersection(coordinates[0], coordinates[1], coordinates[3], coordinates[4])
p2 = get_intersection(coordinates[1], coordinates[2], coordinates[4], coordinates[5])
p3 = get_intersection(coordinates[0], coordinates[3], coordinates[2], coordinates[5])

print(
    max(p1[1], p2[1], p3[1]),
    min(p1[1], p2[1], p3[1]),
    max(p1[0], p2[0], p3[0]),
    min(p1[0], p2[0], p3[0])
)


In [ ]:
plt.figure(figsize=(20, 20))

plt.imshow(image)

for index, coordinate in enumerate(coordinates):
    x, y = coordinate
    letter = 'ABCDEFG'[index]
    title = '{} / {}\n({}, {})'.format(letter, index, x, y)
    plt.scatter(*coordinate, s=100, c='white', marker='o', edgecolors='b')

left = -99999
right = width + 99999
    
angle, c = get_line(coordinates[0], coordinates[1])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[2])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[3], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[4], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

angle, c = get_line(coordinates[0], coordinates[3])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[2], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

plt.ylim(max(p1[1], p2[1], p3[1], height), min(p1[1], p2[1], p3[1], 0))
plt.xlim(min(p1[0], p2[0], p3[0]), max(p1[0], p2[0], p3[0]))

plt.scatter(*p1, s=200, c='blue', marker='o', edgecolors='b')
plt.scatter(*p2, s=200, c='blue', marker='o', edgecolors='b')
plt.scatter(*p3, s=200, c='blue', marker='o', edgecolors='b')

In [ ]:
plt.figure(figsize=(10, 10))

plt.imshow(image)

for index, coordinate in enumerate(coordinates):
    x, y = coordinate
    letter = 'ABCDEFG'[index]
    title = '{} / {}\n({}, {})'.format(letter, index, x, y)
    plt.scatter(*coordinate, s=100, c='white', marker='o', edgecolors='b')

left = -99999
right = width + 99999
    
angle, c = get_line(coordinates[0], coordinates[1])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[2])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[3], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[4], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

angle, c = get_line(coordinates[0], coordinates[3])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[2], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

plt.ylim(max(p1[1], p2[1], p3[1], height), min(p1[1], p2[1], p3[1], 0))
plt.xlim(min(p1[0], p2[0], p3[0]), max(p1[0], p2[0], p3[0]))

#plt.scatter(*p1, s=200, c='blue', marker='o', edgecolors='b')
plt.scatter(*p2, s=200, c='blue', marker='o', edgecolors='b')
plt.scatter(*p3, s=200, c='blue', marker='o', edgecolors='b')

plt.plot([p1[0], p2[0]], [p1[1], p2[1]], c='green')
plt.plot([p2[0], p3[0]], [p2[1], p3[1]], c='green')
plt.plot([p1[0], p3[0]], [p1[1], p3[1]], c='green')

In [ ]:
top_1 = get_intersection(p1, p2, coordinates[1], coordinates[4])
top_2 = get_intersection(p1, p2, coordinates[2], coordinates[5])

left_1 = get_intersection(p1, p3, coordinates[1], coordinates[2])
left_2 = get_intersection(p1, p3, coordinates[4], coordinates[5])

right_1 = get_intersection(p3, p2, coordinates[0], coordinates[1])
right_2 = get_intersection(p3, p2, coordinates[3], coordinates[4])

top_1, top_2, left_1, left_2, right_1, right_2

In [ ]:
plt.figure(figsize=(20, 20))

plt.imshow(image)

for index, coordinate in enumerate(coordinates):
    x, y = coordinate
    letter = 'ABCDEFG'[index]
    title = '{} / {}\n({}, {})'.format(letter, index, x, y)
    plt.scatter(*coordinate, s=100, c='white', marker='o', edgecolors='b')

left = -99999
right = width + 99999
    
angle, c = get_line(coordinates[0], coordinates[1])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[2])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[3], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[4], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

angle, c = get_line(coordinates[0], coordinates[3])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[1], coordinates[4])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')
angle, c = get_line(coordinates[2], coordinates[5])
plt.plot([left, right], [left * angle + c, right * angle + c], c='red')

plt.ylim(max(p1[1], p2[1], p3[1], height), min(p1[1], p2[1], p3[1], 0))
plt.xlim(min(p1[0], p2[0], p3[0]), max(p1[0], p2[0], p3[0]))

plt.scatter(*p1, s=200, c='blue', marker='o', edgecolors='b')
#plt.scatter(*p2, s=200, c='blue', marker='o', edgecolors='b')
#plt.scatter(*p3, s=200, c='blue', marker='o', edgecolors='b')

plt.plot([p1[0], p2[0]], [p1[1], p2[1]], c='green')
plt.plot([p2[0], p3[0]], [p2[1], p3[1]], c='green')
plt.plot([p1[0], p3[0]], [p1[1], p3[1]], c='green')

plt.scatter(*top_1, s=100, c='violet', marker='o', edgecolors='b')
#plt.scatter(*top_2, s=100, c='violet', marker='o', edgecolors='b')
#plt.scatter(*left_1, s=100, c='violet', marker='o', edgecolors='b')
plt.scatter(*left_2, s=100, c='violet', marker='o', edgecolors='b')
#plt.scatter(*right_1, s=100, c='violet', marker='o', edgecolors='b')
plt.scatter(*right_2, s=100, c='violet', marker='o', edgecolors='b')

In [ ]:
multiplier_1 = get_axis_distance_multiplier(
    p2, coordinates[4], coordinates[5], left_2
)
multiplier_2 = get_axis_distance_multiplier(
    p1, coordinates[4], coordinates[3], right_2
)
multiplier_3 = get_axis_distance_multiplier(
    p3, coordinates[4], coordinates[1], top_1
)

multipliers = np.array(
    [multiplier_1, multiplier_2, multiplier_3]
)
    
multiplier_1, multiplier_2, multiplier_3

In [ ]:
coordinates = multipliers * box_size
coordinates

In [ ]:
matplotlib.use('MacOSX')

plt.figure(figsize=(10, 5))
image = imageio.imread('img3.jpg')
_, orig_width, _ = image.shape
print(orig_width)
image = imutils.resize(image, width=1000)

getter = LocationGetter(np.array(all_coordinates[2]) * (1000 / orig_width))
getter.plot(image)
print(getter.get())

In [ ]:
image = imageio.imread('img2.jpg')
getter = LocationGetter(all_coordinates[1])
getter.plot(image)
print(getter.get())

In [ ]:
image = imageio.imread('img1.jpg')
getter = LocationGetter(all_coordinates[0])
getter.plot(image)
print(image.shape)
print(getter.get())

# Creating a video

In [ ]:
# import sys
# !{sys.executable} -m pip install opencv-contrib-python

In [ ]:
#import sys
#!{sys.executable} -m which python




In [ ]:
video_file_name = 'IMG_1104.MOV'


In [ ]:
# video_stream = cv2.VideoCapture('video2.mp4')
# video_file_name = 'IMG_1040.MOV'

video_stream = cv2.VideoCapture(video_file_name)

index = 0

location_frames = []

frame = None

while True:
#for index, (_, frame) in enumerate(video_stream.read):
    _, frame = video_stream.read()

    if frame is None:
        break
    
    if index % 100 == 0:
        print(type(frame))
        print(index, frame.shape)
        location_frames.append((index, frame))
    
    index += 1
    

orig_height, orig_width = 1080, 1920

In [ ]:

thing_index = 0
frame_index, frame = location_frames[thing_index]
frame_coordinates = {}


In [ ]:
# image = imageio.imread('img3.jpg')

# plt.plot(figsize=(10, 10))

ax = plt.gca()
fig = plt.gcf()
implot = ax.imshow(np.flip(frame, 2))

coordinates = []

def onclick(event):
    #print('onclick')
    if event.xdata != None and event.ydata != None:
        global coordinates
        global frame_coordinates
        global frame_index
        global frame
        global thing_index
        
        coordinates.append((event.xdata, event.ydata))
        
        if len(coordinates) == 6:
            frame_coordinates[frame_index] = coordinates
            thing_index += 1
            frame_index, frame = location_frames[thing_index]
            coordinates = []
            implot = ax.imshow(np.flip(frame, 2))

        
        print('added', len(coordinates), coordinates)
        #print(event.xdata, event.ydata)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()


In [ ]:
frame_coordinates

In [ ]:
frame_coordinates = {0: [(589.4300699300697, 412.72677322677305),
  (804.1353646353643, 441.85764235764225),
  (1195.7837162837159, 436.46303696303687),
  (595.9035964035963, 554.0654345654344),
  (805.2142857142856, 677.0624375624375),
  (1180.678821178821, 614.4850149850149)],
 100: [(621.7977022977022, 328.57092907092897),
  (830.0294705294702, 243.33616383616368),
  (1237.8616383616381, 324.2552447552446),
  (621.7977022977022, 470.98851148851134),
  (828.9505494505493, 503.3561438561437),
  (1222.756743256743, 511.9875124875124)],
 200: [(602.3771228771227, 586.4330669330668),
  (794.4250749250748, 603.695804195804),
  (1168.810689310689, 578.8806193806192),
  (605.6138861138859, 723.4560439560439),
  (800.8986013986012, 830.2692307692307),
  (1174.2052947052944, 749.3501498501498)],
 300: [(548.4310689310688, 448.3311688311687),
  (724.2952047952047, 458.0414585414584),
  (959.4999999999998, 459.1203796203795),
  (552.7467532467531, 548.670829170829),
  (726.4530469530469, 587.5119880119879),
  (956.2632367632366, 569.1703296703296)],
 400: [(553.8256743256742, 423.5159840159839),
  (835.4240759240756, 441.85764235764225),
  (945.4740259740258, 445.09440559440543),
  (555.9835164835164, 521.697802197802),
  (830.0294705294702, 551.9075924075922),
  (942.2372627372624, 533.5659340659339)],
 500: [(740.4790209790208, 367.4120879120877),
  (1069.5499500499498, 377.1223776223775),
  (1243.2562437562435, 376.0434565434564),
  (745.8736263736262, 488.25124875124857),
  (1060.9185814185812, 540.0394605394604),
  (1231.3881118881116, 504.4350649350648)],
 600: [(931.4480519480517, 304.83466533466515),
  (1139.67982017982, 304.83466533466515),
  (1631.6678321678316, 292.9665334665333),
  (936.842657342657, 440.77872127872115),
  (1134.2852147852145, 550.8286713286712),
  (1593.9055944055938, 513.0664335664335)],
 700: [(327.25224775224774, 332.88661338661325),
  (969.2102897102895, 356.62287712287696),
  (1156.9425574425572, 358.78071928071915),
  (356.3831168831168, 546.5129870129869),
  (959.4999999999998, 661.9575424575423),
  (1140.7587412587409, 536.8026973026972)],
 800: [(349.90959040959035, 522.7767232767231),
  (940.0794205794205, 546.5129870129869),
  (1061.9975024975022, 524.9345654345653),
  (357.4620379620379, 711.587912087912),
  (933.6058941058939, 786.0334665334665),
  (1058.760739260739, 679.2202797202796)],
 900: [(376.88261738261735, 470.98851148851134),
  (961.657842157842, 528.1713286713285),
  (1078.1813186813183, 464.51498501498486),
  (398.46103896103887, 653.3261738261737),
  (948.710789210789, 747.1923076923076),
  (1065.2342657342654, 619.8796203796203)]}

frame_coordinates = {0: [(376.2432567432567, 206.65284715284724),
  (641.657842157842, 278.5809190809191),
  (749.54995004995, 238.3011988011989),
  (429.4700299700299, 478.5409590409591),
  (638.0614385614385, 618.0814185814186),
  (722.2172827172826, 533.9255744255745)],
 100: [(434.504995004995, 159.89960039960044),
  (745.2342657342656, 143.35614385614394),
  (815.0044955044953, 179.32017982017987),
  (437.3821178821178, 480.6988011988013),
  (710.7087912087911, 585.7137862137863),
  (780.4790209790208, 523.1363636363637)],
 200: [(468.31118881118874, 321.7377622377623),
  (713.585914085914, 362.7367632367633),
  (759.61988011988, 336.8426573426574),
  (479.1003996003996, 551.1883116883117),
  (696.3231768231767, 635.3441558441559),
  (738.760739260739, 572.0474525474526)]}


In [ ]:
vid1_coordinates = np.array([(776.2274144820192 + 2, 413.70061397524614 + 2),
 (1205.035522853523, 425.3953805671962),
 (1401.8974271513496, 403.9549751486211),
 (776.2274144820192, 688.5276288860734),
 (1197.2390117922228, 733.3575674885489),
 (1390.2026605593994, 641.748562518273)])

vid2_coordinates = np.array([(540.8598166539342, 491.23414820473636),
 (847.9644766997708, 521.7918258212376),
 (1359.805576776165, 526.3754774637127),
 (539.3319327731091, 689.8590527119939),
 (844.9087089381205, 798.3388082505729),
 (1347.5825057295644, 763.1974789915967)]
)

vid3_coordinates = np.array([(588.3511488511488, 413.80569430569415),
 (805.2142857142856, 442.93656343656335),
 (1194.704795204795, 436.46303696303687),
 (594.8246753246751, 552.9865134865133),
 (805.2142857142856, 674.9045954045953),
 (1180.678821178821, 614.4850149850149)]
)

orig_coordinates = vid3_coordinates

In [ ]:
#video_width = orig_width
half_box_size = 15

# coordinates = orig_coordinates * video_width / orig_width

video_stream = cv2.VideoCapture(video_file_name)
_, frame = video_stream.read()
#frame = imutils.resize(frame, width=video_width)

trackers = []

    
tracker_locations = []

frame_index = 0
while True:
    
    if frame_index in frame_coordinates:
        tracker_coordinates = frame_coordinates[frame_index]
        trackers = []
        for coordinate_index, c in enumerate(tracker_coordinates):
            #tracker = cv2.TrackerMOSSE_create() # Fastest not accurate
            #tracker = cv2.TrackerKCF_create() # Middle ground
            tracker = cv2.TrackerCSRT_create() # Slowest most accurate
            tracker.init(
                frame,
                (
                    # Middle points
                    c[0] - half_box_size,
                    c[1] - half_box_size,
                    2 * half_box_size,
                    2 * half_box_size
                )
                if True else ( # coordinate_index in [1, 4] else (
                    (
                        # Left points
                        c[0] - 7, # - half_box_size,
                        c[1] - half_box_size,
                        2 * half_box_size,
                        2 * half_box_size
                    )
                    if coordinate_index in [0, 3] else
                    (
                        # Right points
                        c[0] - 2 * half_box_size + 7,
                        c[1] - half_box_size,
                        2 * half_box_size,
                        2 * half_box_size
                    )
                )
            )
            trackers.append(tracker)

    
    
    _, frame = video_stream.read()
    #frame = imutils.resize(frame, width=video_width)

    if frame is None:
        break
    
    locations = []
    for tracker_index, tracker in enumerate(trackers):
        tracker_success, tracker_box = tracker.update(frame)
        if tracker_success:
            #print(tracker_box)
            x, y, w, h = np.array(tracker_box).astype('int')
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
            if True: # tracker_index in [1, 4]:
                locations.append(((x + x + w) / 2, (y + y + h) / 2))
            elif tracker_index in [0, 3]:
                locations.append((x, (y + y + h) / 2))
            else:
                locations.append(((x + w), (y + y + h) / 2))
        else:
            print('boo')
            locations.append((0,0))
    tracker_locations.append(locations)
    
    cv2.imshow('Frame', frame)

    
    #print(tracker_success, tracker_box)
    #break
    
    #frame = np.flip(frame, 2)
    
    
    key = cv2.waitKey(1) & 0xFF

    if key == ord('r'):
        video_stream = cv2.VideoCapture('video.mp4')
    if key == ord('s'):
        pass
    if key == ord('q'):
        break
    
    frame_index += 1
    
video_stream.release()
cv2.destroyAllWindows()

In [ ]:
video_stream.release()
cv2.destroyAllWindows()

In [ ]:
np.array(tracker_locations).shape

In [ ]:
#video_stream = cv2.VideoCapture(video_file_name)
#
#for i in range(910):
#    _, frame = video_stream.read()
#    
#    if frame.shape != (1080, 1920, 3):
#        print(i, frame.shape)


In [ ]:
# tracker_locations

In [ ]:
# video_stream = cv2.VideoCapture(video_file_name)
# _, frame = video_stream.read()

import matplotlib
matplotlib.use('Agg')

#video_width = 1000

video_stream = cv2.VideoCapture(video_file_name)

height, width, _ = (1046, 1860, None)

#fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
fourcc = cv2.VideoWriter_fourcc(*'XVID')

#width, height = 1046, 1860
height, width = 1046, 1860

print('width and height', width, height)
output_video = cv2.VideoWriter(
    filename='output_video.mp4',
    apiPreference=cv2.CAP_FFMPEG,
    fourcc=fourcc,
    fps=30,
    frameSize=(width,height),
)
#output_video = cv2.VideoWriter('output_video.avi', -1, 1, (width,height))

all_coordinates = []

frame = None

for i in range(len(tracker_locations)):
    plt.figure(figsize=(12, 10))

    plt.gca().set_axis_off()
    
    plt.box(False)
    
    _, frame = video_stream.read()
    
    if frame is None:
        break
    
    file_name = 'plots/fig.png'
    #print(file_name)
    
    getter = LocationGetter(np.array(tracker_locations[i]))# * video_width / orig_video_width)
    getter.plot(np.flip(frame, 2))
    
    all_coordinates.append(getter.real_life_coordinates)
    
    plt.savefig(file_name, bbox_inches='tight', pad_inches = 0, dpi = 200)
    plt.close('all')
    
    old_frame = frame
    output_frame = imageio.imread(file_name)
    
    frame = output_frame[:,:,0:3]
    frame = np.flip(frame, 2)
    
    print(i, output_frame.shape, frame.shape, old_frame.shape if old_frame is not None else None)
    
    if frame.shape != (height, width, 3):
        print('WTF!@!@!@!!!!')
        frame = old_frame
    
    old_frame = frame

    if not frame.shape == (height, width, 3):
        continue
        print('REMOVE ME')
    
    assert frame.shape == (height, width, 3)
    
    output_video.write(frame)
    
    #if i == 20:
    #    break
    #if i == 30 * 10 - 1:
    #    break

all_coordinates = np.array(all_coordinates)
        
output_video.release()
video_stream.release()

print(' ' + '*' * 20)
print(' * DONE')
print(' ' + '*' * 20)


In [ ]:
output_video.release()
video_stream.release()
all_coordinates = np.array(all_coordinates)

In [ ]:
all_coordinates.shape

In [ ]:
# np.savez_compressed('all_coordinates.npz', all_coordinates=all_coordinates)


## Plot 3d coordinates of the camera

In [ ]:
all_coordinates = np.load('all_coordinates.npz')['all_coordinates']

In [ ]:
np.array(all_coordinates).shape

In [ ]:
#ax.scatter(*all_coordinates[10], color='blue', alpha=1, marker='o')

#ax.scatter(*all_coordinates.transpose(), color='blue', alpha=0.5, marker='.')


a, b, c = 1.905, 2.52, 1.115

box_corners = np.array([
    (0, 0, 0),
    (a, 0, 0),
    (a, b, 0),
    (a, b, c),
    (a, 0, c),
    (0, b, c),
    (0, 0, c),
])



box_lines = np.array([
    (
        (0, 0, 0),
        (a, 0, 0),
    ),
    (
        (0, 0, c),
        (a, 0, c),
    ),
    (
        (0, b, 0),
        (a, b, 0),
    ),
    (
        (0, b, c),
        (a, b, c),
    ),
    (
        (0, b, 0),
        (0, b, c),
    ),
    (
        (a, b, 0),
        (a, b, c),
    ),
    (
        (0, 0, 0),
        (0, 0, c),
    ),
    (
        (a, 0, 0),
        (a, 0, c),
    ),
    (
        (0, 0, 0),
        (0, b, 0),
    ),
    (
        (a, 0, 0),
        (a, b, 0),
    ),
    (
        (0, 0, c),
        (0, b, c),
    ),
    (
        (a, 0, c),
        (a, b, c),
    ),
])
fig = plt.figure(figsize=(7, 12))


ax = fig.add_subplot(3, 1, 1, projection='3d')
ax.scatter(*(box_corners + [a, b, 0]).transpose(), color='red', marker='x')
ax.scatter(*all_coordinates.transpose(), color='lightblue', alpha=0.2, marker='.')
ax.view_init(10, 90)
for point_1, point_2 in box_lines:
    ax.plot(*(np.array([point_1, point_2]) + [a, b, 0]).transpose(), color='orange')
    
ax = fig.add_subplot(3, 1, 2, projection='3d')
ax.scatter(*(box_corners + [a, b, 0]).transpose(), color='red', marker='x')
ax.scatter(*all_coordinates.transpose(), color='lightblue', alpha=0.2, marker='.')
ax.view_init(10, 2 * 90)
for point_1, point_2 in box_lines:
    ax.plot(*(np.array([point_1, point_2]) + [a, b, 0]).transpose(), color='orange') 
    
ax = fig.add_subplot(3, 1, 3, projection='3d')
ax.scatter(*(box_corners + [a, b, 0]).transpose(), color='red', marker='x')
ax.scatter(*all_coordinates.transpose(), color='lightblue', alpha=0.2, marker='.')
ax.view_init(90, 0)
for point_1, point_2 in box_lines:
    ax.plot(*(np.array([point_1, point_2]) + [a, b, 0]).transpose(), color='orange')

In [ ]:
# ffmpeg -f image2 -i ./plots/fig_%01d.png -b 200k -vcodec mpeg4 -y ./output.mp4

In [ ]:
# ffmpeg -i ./plots/fig_%01d.png -vcodec h264 -b 10M -crf 18 -y ./output.mp4

In [ ]:
[d for d in dir(plt) if 'close' in d.lower()]

In [ ]:
output_video.release()

In [ ]:
getter.real_life_coordinates

In [ ]:
print(output_frame.shape)
plt.imshow(output_frame[:,:,0])